In [ ]:
from train import *

# Параметры
sharp_dir = "data/sharp_train"  # тут лежат резкие фото
val_dir   = "data/sharp_val"    # валидация (резкие)
crop = 256
batch_size = 8
epochs = 20
lr = 2e-4
device = "cuda" if torch.cuda.is_available() else "cpu"

train_ds = DeblurSyntheticDataset(sharp_dir, crop_size=crop)
val_ds   = DeblurSyntheticDataset(val_dir,   crop_size=crop)

train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl   = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

model = UNetSmall().to(device)
optim = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
loss_fn = DeblurLoss(w_l1=1.0, w_ssim=0.5)

best_psnr = -1
save_path = "deblur_unet.pth"

for ep in range(1, epochs+1):
    model.train()
    total = 0.0
    for blurred, sharp, _ in train_dl:
        blurred = blurred.to(device)
        sharp   = sharp.to(device)

        pred = model(blurred)
        loss = loss_fn(pred, sharp)

        optim.zero_grad()
        loss.backward()
        optim.step()

        total += loss.item() * blurred.size(0)

    train_loss = total / len(train_dl.dataset)

    # валидация
    model.eval()
    val_psnrs, val_ssims = [], []
    with torch.no_grad():
        for blurred, sharp, _ in val_dl:
            blurred = blurred.to(device)
            sharp   = sharp.to(device)
            pred = model(blurred)
            val_psnrs.append(psnr(pred, sharp))
            val_ssims.append(ssim_simple(pred, sharp))
    val_psnr = torch.cat(val_psnrs).mean().item()
    val_ssim = torch.cat(val_ssims).mean().item()

    if val_psnr > best_psnr:
        best_psnr = val_psnr
        torch.save({'model': model.state_dict(),
                    'psnr': best_psnr}, save_path)

    print(f"Epoch {ep:02d}/{epochs} | train_loss={train_loss:.4f} | "
          f"val_PSNR={val_psnr:.2f} | val_SSIM={val_ssim:.3f} | best={best_psnr:.2f}")

print(f"Лучшая модель сохранена в: {save_path}")


In [ ]:
@torch.no_grad()
def deblur_image(model, img_pil, tile=None):
    """Прямой прогон. Если картинка большая, можно тайлить (tile=512, например)."""
    model.eval()
    x = torch.from_numpy(np.asarray(img_pil.convert("RGB"), np.float32)/255.).permute(2,0,1).unsqueeze(0).to(device)
    if tile is None:
        y = model(x).clamp(0,1)
        out = (y.squeeze(0).permute(1,2,0).cpu().numpy()*255).astype(np.uint8)
        return Image.fromarray(out)
    else:
        # простая тайловая схема с перекрытием
        b, c, h, w = x.shape
        t = tile
        ov = t//8
        out = torch.zeros_like(x)
        weight = torch.zeros_like(x)
        for i in range(0, h, t-ov):
            for j in range(0, w, t-ov):
                hs, ws = min(t, h - i), min(t, w - j)
                patch = x[..., i:i+hs, j:j+ws]
                pred = model(patch).clamp(0,1)
                out[..., i:i+hs, j:j+ws] += pred
                weight[..., i:i+hs, j:j+ws] += 1.0
        y = out / (weight + 1e-8)
        out_img = (y.squeeze(0).permute(1,2,0).cpu().numpy()*255).astype(np.uint8)
        return Image.fromarray(out_img)

# Загрузка чекпойнта и прогон папки
ckpt = torch.load("deblur_unet.pth", map_location=device)
model.load_state_dict(ckpt['model'])

in_dir  = "data/blurred_eval"    # можно реальные смазанные фото
out_dir = Path("results"); out_dir.mkdir(exist_ok=True)

for p in sorted(glob.glob(f"{in_dir}/*.*")):
    im = Image.open(p).convert("RGB")
    res = deblur_image(model, im, tile=None)  # или tile=512
    res.save(out_dir / (Path(p).stem + "_deblur.png"))
